Copyright (C) Microsoft Corporation.  
see license file for details

### Download all data from https://nihcc.app.box.com/v/ChestXray-NIHCC 
 - Download 12 compressed (.tar.gz) files, plus two meta info files (Data_Entry_2017.csv and BBox_List_2017.csv). 
 - Uncompress the 12 .tar.gz files to generate 112120 .png images (e.g. 00000001_000.png)  
 - uploads local data to Azure Data Storage containers (see in cell below the local and Azure storage containers structure)

 

   

In [ ]:
# Local data structure:
#     /data_dir/chestxray/ChestX-ray8/  
#         BBox_List_2017.csv         
#         Data_Entry_2017.csv  
#         blacklist.csv
#         ChestXray-NIHCC/  
#             /tmp/  
#                 images_01.tar.gz    
#                 images_03.tar.gz    
#                 images_05.tar.gz   
#                 images_07.tar.gz   
#                 images_09.tar.gz   
#                 images_11.tar.gz  
#                 images_02.tar.gz   
#                 images_04.tar.gz   
#                 images_06.tar.gz   
#                 images_08.tar.gz   
#                 images_10.tar.gz   
#                 images_12.tar.gz
#             /images/
#                   00000001_000.png
#                   00000001_001.png
#                   ... 112120 png images
# 
#    Azure Data Storage container structure:
#       container nihchestxrayimages
#                   00000001_000.png
#                   00000001_001.png
#                   ... 112120 png image files, total 41.96 GiB
#       container nihchestxraydatacompressed:
#                   images_01.tar.gz
#                   ... 12 files, total 41.98 GiB
#       container nihchestxraydata
#                   BBox_List_2017.csv
#                   blacklist.csv
#                   Data_Entry_2017.csv

In [1]:
# Allow multiple displays per cell

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
do_compressed_data_FLAG = True

In [3]:
import os, platform
import sys
import urllib.request
import shutil

In [4]:
%load_ext dotenv
dotenv_file_path='./../../not_shared/general.env'

In [5]:
platform.platform()
os.getcwd()

'Linux-4.15.0-1063-azure-x86_64-with-debian-stretch-sid'

'/datadrive01/prj/AzureChestXRayNoAML/code/01_DataPrep'

In [6]:
# import utlity functions

paths_to_append = [os.path.join(os.getcwd(), os.path.join(*(['..','src'])))]
paths_to_append
def add_path_to_sys_path(path_to_append):
    if not (any(path_to_append in paths for paths in sys.path)):
        sys.path.append(path_to_append)
[add_path_to_sys_path(crt_path) for crt_path in paths_to_append]

import azure_chestxray_utils

['/datadrive01/prj/AzureChestXRayNoAML/code/01_DataPrep/../src']

[None]

/data/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [7]:
# create the file path variables 
# create nih_chest_xray_data_dir (in container dir mapped to a host dir for data persistence), 
# where data will be copied from blob

prj_consts = azure_chestxray_utils.chestxray_consts()

nih_chest_xray_data_dir=os.path.join(os.path.join(*([os.sep]+prj_consts.BASE_INPUT_DIR_list)), 
                                     os.path.join(*(prj_consts.ChestXray_IMAGES_DIR_list)))
os.makedirs(nih_chest_xray_data_dir, exist_ok=True)

nih_chest_xray_data_dir
!ls -l {nih_chest_xray_data_dir}

'/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC'

total 4
drwxrwxrwx 2 root root 4096 Nov 16 05:48 tmp


###### Download NIH data from https://nihcc.app.box.com/v/ChestXray-NIHCC

In [8]:
tmp_dir = os.path.join(nih_chest_xray_data_dir, 'tmp')
tmp_dir
os.makedirs(tmp_dir, exist_ok=True)

'/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp'

In [9]:
# https://nihcc.app.box.com/v/ChestXray-NIHCC/file/371647823217
# batch_download_zips.py

# Download the 56 zip files in Images_png in batches

# URLs for the zip files
links = [
    'https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz',
    'https://nihcc.box.com/shared/static/i28rlmbvmfjbl8p2n3ril0pptcmcu9d1.gz',
    'https://nihcc.box.com/shared/static/f1t00wrtdk94satdfb9olcolqx20z2jp.gz',
	'https://nihcc.box.com/shared/static/0aowwzs5lhjrceb3qp67ahp0rd1l1etg.gz',
    'https://nihcc.box.com/shared/static/v5e3goj22zr6h8tzualxfsqlqaygfbsn.gz',
	'https://nihcc.box.com/shared/static/asi7ikud9jwnkrnkj99jnpfkjdes7l6l.gz',
	'https://nihcc.box.com/shared/static/jn1b4mw4n6lnh74ovmcjb8y48h8xj07n.gz',
    'https://nihcc.box.com/shared/static/tvpxmn7qyrgl0w8wfh9kqfjskv6nmm1j.gz',
	'https://nihcc.box.com/shared/static/upyy3ml7qdumlgk2rfcvlb9k6gvqq2pj.gz',
	'https://nihcc.box.com/shared/static/l6nilvfa9cg3s28tqv1qc1olm3gnz54p.gz',
	'https://nihcc.box.com/shared/static/hhq8fkdgvcari67vfhs7ppg2w6ni4jze.gz',
	'https://nihcc.box.com/shared/static/ioqwiy20ihqwyr8pf4c24eazhh281pbu.gz'
]

for idx, link in enumerate(links):
    fn = 'images_%02d.tar.gz' % (idx+1)
    fn = os.path.join(tmp_dir, fn)
    print( 'downloading {} to {} ...'.format(link, fn))
    if do_compressed_data_FLAG:
        urllib.request.urlretrieve(link, fn)  
print ("Download complete. Please check the checksums")

downloading https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz to /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_01.tar.gz ...


('/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_01.tar.gz',
 <http.client.HTTPMessage at 0x7fdcfd0ba160>)

downloading https://nihcc.box.com/shared/static/i28rlmbvmfjbl8p2n3ril0pptcmcu9d1.gz to /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_02.tar.gz ...


('/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_02.tar.gz',
 <http.client.HTTPMessage at 0x7fdcfd0ba550>)

downloading https://nihcc.box.com/shared/static/f1t00wrtdk94satdfb9olcolqx20z2jp.gz to /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_03.tar.gz ...


...

downloading https://nihcc.box.com/shared/static/hhq8fkdgvcari67vfhs7ppg2w6ni4jze.gz to /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_11.tar.gz ...


('/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_11.tar.gz',
 <http.client.HTTPMessage at 0x7fdcfd0ba2e8>)

downloading https://nihcc.box.com/shared/static/ioqwiy20ihqwyr8pf4c24eazhh281pbu.gz to /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_12.tar.gz ...


('/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_12.tar.gz',
 <http.client.HTTPMessage at 0x7fdcfd0ba908>)

Download complete. Please check the checksums


###### Cell below (Commented) should __not__ be run
Instead manually click on corresponding files at app.box.com and download them manually

In [10]:
# #NIH ChestXray data at app.box.com
# # https://nihcc.app.box.com/v/ChestXray-NIHCC/folder/36938765345
# #click on Data_Entry_2017.csv and 'BBox_List_2017.csv' to see their links
# tmp_dir1 = os.path.join(nih_chest_xray_data_dir, '..')
# link = 'https://nihcc.app.box.com/v/ChestXray-NIHCC/file/219760887468'
# fn = 'Data_Entry_2017.csv'
# fn = os.path.join(tmp_dir1, fn)
# print( 'downloading {} to {} ...'.format(link, fn))
# urllib.request.urlretrieve(link, fn)


# link = 'https://nihcc.app.box.com/v/ChestXray-NIHCC/file/219760940956'
# fn = 'BBox_List_2017.csv'
# fn = os.path.join(tmp_dir1, fn)
# print( 'downloading {} to {} ...'.format(link, fn))
# urllib.request.urlretrieve(link, fn)

downloading https://nihcc.app.box.com/v/ChestXray-NIHCC/file/219760887468 to /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/../Data_Entry_2017.csv ...


('/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/../Data_Entry_2017.csv',
 <http.client.HTTPMessage at 0x7fdcfd0ba198>)

downloading https://nihcc.app.box.com/v/ChestXray-NIHCC/file/219760940956 to /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/../BBox_List_2017.csv ...


('/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/../BBox_List_2017.csv',
 <http.client.HTTPMessage at 0x7fdcfd0ba438>)

In [11]:
!pwd
! ls -l ./data/blacklist.csv
# finally copy Azure chest Xray blacklist.csv to the same location
shutil.copy('./data/blacklist.csv', tmp_dir1)

/datadrive01/prj/AzureChestXRayNoAML/code/01_DataPrep
-rw-rw-r-- 1 loginvm022 loginvm022 20808 Nov 16 23:57 ./data/blacklist.csv


'/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/../blacklist.csv'

In [12]:
tmp_dir
[f for f in os.listdir(tmp_dir1) if os.path.isfile(os.path.join(tmp_dir, f))]
tmp_dir1
[f for f in os.listdir(tmp_dir1) if os.path.isfile(os.path.join(tmp_dir1, f))]


'/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp'

[]

'/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/..'

['Data_Entry_2017.csv', 'blacklist.csv', 'BBox_List_2017.csv']

###### Upload Downloaded NIH data to Azure blob storage, in two containers:
 - one for compressed data (__nihchestxraydatacompressed__ i.e. os.getenv('sa_container_name_compressed_data')) 
 - one for regular data (__nihchestxraydata__ i.e. os.getenv('sa_container_name'))

In [13]:
az_copy_version = !azcopy --help
az_copy_version[:3]

['------------------------------------------------------------------------------',
 'azcopy 7.2.0-netcore Copyright (c) 2018 Microsoft Corp. All Rights Reserved.',
 '------------------------------------------------------------------------------']

In [14]:
%dotenv $dotenv_file_path -o

crt_container = 'https://'+os.getenv('sa_name')+'.blob.core.windows.net/'+os.getenv('sa_container_name_compressed_data')
crt_container

'https://ghiordanazrchestxraysa.blob.core.windows.net/nihchestxraydatacompressed'

In [15]:
def upload_folder_using_azcopy(local_folder, to_container=crt_container):
    image_files = [f for f in os.listdir(local_folder) if os.path.isfile(os.path.join(local_folder, f))]

    def upload_file(file_to_upload, crt_local_folder=local_folder, to_container=to_container):
        cli_command = 'azcopy ' +\
        '--source ' + crt_local_folder + ' ' + \
        '--destination ' + to_container + ' ' + \
        '--include ' + file_to_upload + ' '
        ! echo $cli_command
        cli_command = cli_command + ' --dest-key ' + os.getenv('sa_key')
        !echo Yes | $cli_command
    [upload_file(x) for x in image_files]

if do_compressed_data_FLAG:
    upload_folder_using_azcopy('/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp')
    
upload_folder_using_azcopy('/data_dir/chestxray/ChestX-ray8', 
                          to_container = \
                        'https://'+os.getenv('sa_name')+'.blob.core.windows.net/'+os.getenv('sa_container_name'))

azcopy --source /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp --destination https://ghiordanazrchestxraysa.blob.core.windows.net/nihchestxraydatacompressed --include images_08.tar.gz
=Overwrite https://ghiordanazrchestxraysa.blob.core.windows.net/nihchestxraydatacompressed/images_08.tar.gz with /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_08.tar.gz? (Yes/No/All) Y
[2019/11/17 20:09:39] Transfer summary:
-----------------
Total files transferred: 1
Transfer successfully:   1
Transfer skipped:        0
Transfer failed:         0
Elapsed time:            00.00:00:15
azcopy --source /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp --destination https://ghiordanazrchestxraysa.blob.core.windows.net/nihchestxraydatacompressed --include images_04.tar.gz
=Overwrite https://ghiordanazrchestxraysa.blob.core.windows.net/nihchestxraydatacompressed/images_04.tar.gz with /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_04.tar.gz? (Yes/No/All) Y
[2019/11/17 20:09:51]

azcopy --source /data_dir/chestxray/ChestX-ray8 --destination https://ghiordanazrchestxraysa.blob.core.windows.net/nihchestxraydata --include BBox_List_2017.csv
=Overwrite https://ghiordanazrchestxraysa.blob.core.windows.net/nihchestxraydata/BBox_List_2017.csv with /data_dir/chestxray/ChestX-ray8/BBox_List_2017.csv? (Yes/No/All) Y
[2019/11/17 20:12:40] Transfer summary:
-----------------
Total files transferred: 1
Transfer successfully:   1
Transfer skipped:        0
Transfer failed:         0
Elapsed time:            00.00:00:00


In [16]:
nih_chest_xray_data_dir
tmp_dir
!ls -l $tmp_dir

'/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC'

'/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp'

total 44023372
-rwxrwxrwx 1 root root 2008470987 Nov 17 19:32 images_01.tar.gz
-rwxrwxrwx 1 root root 3952623504 Nov 17 19:36 images_02.tar.gz
-rwxrwxrwx 1 root root 3929234850 Nov 17 19:39 images_03.tar.gz
-rwxrwxrwx 1 root root 3838903983 Nov 17 19:42 images_04.tar.gz
-rwxrwxrwx 1 root root 3935496531 Nov 17 19:45 images_05.tar.gz
-rwxrwxrwx 1 root root 3986301172 Nov 17 19:48 images_06.tar.gz
-rwxrwxrwx 1 root root 4016328426 Nov 17 19:52 images_07.tar.gz
-rwxrwxrwx 1 root root 4018347353 Nov 17 19:56 images_08.tar.gz
-rwxrwxrwx 1 root root 4111327929 Nov 17 19:59 images_09.tar.gz
-rwxrwxrwx 1 root root 4181556296 Nov 17 20:02 images_10.tar.gz
-rwxrwxrwx 1 root root 4187084020 Nov 17 20:07 images_11.tar.gz
-rwxrwxrwx 1 root root 2914187733 Nov 17 20:09 images_12.tar.gz


The unzipping process below will run for several minutes.

While it runs, you can monitor the progress by ssh-ing 
into your compute context machine and running:   
find /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/images  -type f | wc -l  

The final number of images is:  
__112120__   



In [17]:
nih_chest_xray_data_dir
old_crt_dir = os.getcwd()
old_crt_dir

'/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC'

'/datadrive01/prj/AzureChestXRayNoAML/code/01_DataPrep'

In [18]:
os.chdir(nih_chest_xray_data_dir)
!pwd

/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC


In [19]:
allImages = ! cat $tmp_dir/*.tar.gz | tar -zxf - -i

In [20]:
# upload_folder_using_azcopy(nih_chest_xray_data_dir, 
#                           to_container = \
#                         'https://'+os.getenv('sa_name')+'.blob.core.windows.net/'+os.getenv('sa_container_name_images'))

images_dir = os.path.join(nih_chest_xray_data_dir, 'images')
cli_command = 'azcopy --quiet ' +\
    ' --source ' + images_dir + ' ' +\
    ' --destination ' + 'https://'+os.getenv('sa_name')+'.blob.core.windows.net/'+os.getenv('sa_container_name_images') + ' ' +\
    ' --recursive ' 

cli_command
cli_command =cli_command+ ' --dest-key ' + os.getenv('sa_key') 
!$cli_command


'azcopy --quiet  --source /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/images  --destination https://ghiordanazrchestxraysa.blob.core.windows.net/nihchestxrayimages  --recursive '

=Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 369.002 KB; Average Speed:39.19 KB/s.                     Finished: 816 file(s), 312.251 MB; Average Speed:27.25 MB/s.                   Finished: 1297 file(s), 498.155 MB; Average Speed:36.88 MB/s.                  Finished: 1992 file(s), 762.677 MB; Average Speed:49.03 MB/s.                  Finished: 1993 file(s), 762.677 MB; Average Speed:43.33 MB/s.                  Finished: 1994 file(s), 762.677 MB; Average Speed:38.81 MB/s.                  Finished: 1994 file(s), 763.056 MB; Average Speed:28.76 MB/s.                  Finished: 2841 file(s), 1.063 GB; Average Speed:38.07 MB/s.                    Finished: 3087 file(s), 1.155 GB; Average Speed:38

Finished: 52410 file(s), 19.613 GB; Average Speed:100.67 MB/s.                 Finished: 53394 file(s), 19.982 GB; Average Speed:101.52 MB/s.                 Finished: 54092 file(s), 20.244 GB; Average Speed:101.81 MB/s.                 Finished: 55274 file(s), 20.687 GB; Average Speed:102.99 MB/s.                 Finished: 56540 file(s), 21.159 GB; Average Speed:104.31 MB/s.                 Finished: 57680 file(s), 21.588 GB; Average Speed:105.39 MB/s.                 Finished: 58772 file(s), 21.997 GB; Average Speed:106.34 MB/s.                 Finished: 58896 file(s), 22.043 GB; Average Speed:105.55 MB/s.                 Finished: 59197 file(s), 22.155 GB; Average Speed:105.08 MB/s.                 Finished: 60201 file(s), 22.532 GB; Average Speed:105.86 MB/s.                 Finished: 60881 file(s), 22.787 GB; Average Speed:106.03 MB/s.                 Finished: 62260 file(s), 23.304 GB; Average Speed:107.44 MB/s.                 Finished: 62985 file(s), 23.576 GB; Average Speed:10

In [21]:
!pwd
os.chdir(old_crt_dir)
!pwd

/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC
/datadrive01/prj/AzureChestXRayNoAML/code/01_DataPrep
